#  赛题十四 —— 论文复现自验报告

赛题内容：在 MindQuantum 框架下利用迁移学习来缓解贫瘠高原现象

论文题目：A Parameter Initialization Method for Variational Quantum Algorithms to Mitigate Barren Plateaus Based on Transfer Learning
 
复现要求：利用 MindQuantum 来实现论文中提到的迁移学习，达到缓解参数化量子线路梯度贫瘠高原现象，复现 Fig3 中不超过12比特的部分。

## 赛题介绍

在采用变分量子算法来求解问题时，需要求解参数对期望值的导数。随着线路的比特数和深度的增加，如果线路参数采用随机初始化，会导致参数梯度指数减小 — 这种现象被称为 “贫瘠高原”。贫瘠高原的出现会极大增加所需迭代次数，从而增大求解成本。如何解决贫瘠高原自然也就成了一个热门话题。

本论文中，提出采用类似经典机器学习中的迁移学习中的量子迁移学习方式，来解决量子变分算法中的贫瘠高原问题。方法的主要思路是将在小尺寸线路（此处称为 “基网络”）上得到的最优参数，连同结构，通过某种方式一起赋予大尺寸线路（此处称为 “目标网络”），从而避免随机初始化。通过实践检验，这种方法对贫瘠高原现象具有一定的缓解效果，对于解决大尺寸量子变分算法求解问题具有重要意义。

本论文共涉及 3 种物理模型：

1. 横场伊辛模型的哈密顿量为：
$$H=-J\Sigma_{i=1}^{N}\sigma_i^z\sigma_{i+1}^z-h\Sigma_{i=1}^N\sigma_i^z.$$
其中，$\sigma_i^\alpha$ 为作用在第 $i$ 个比特上的泡利算符。$J$ 为最近邻自旋的耦合强度，此处设 $J=1$ 并将其作为系统的能量单位。系统满足周期性边界条件。而 $h=2$。
基网络设为 4 比特 4 层的硬件高效拟设（HEA），而目标网络为 6 - 12 比特的硬件高效拟设线路。硬件高效拟设线路中，每一层由作用在每个比特上由连续 `RZ`、`RX`、`RZ` 参数门构成的任意单比特操作和所有相邻比特间的双比特纠缠门 `CZ` 门构成。

2. 氢链模型的哈密顿量为：
$$H=\Sigma_{pq}h_{pq}a_p^\dagger a_q + \Sigma_{pqrs}V_{pqrs}a_p^\dagger a_q^\dagger a_r a_s.$$
其中，$a_p^\dagger(a_p)$ 表示产生（湮灭）算符，$h_{pq}$ 和 $V_{pqrs}$ 分别表示单和双电子积分。考虑线性结构，氢原子之间的原子距离为 0.74 A。 采用 STO-3G 基底和 Jordan-Wigner 变换。对于 $m$ 个原子，需要 $2m$ 个比特的哈密顿量来表述，其可以表示为泡利算符的线性组合 $H=\Sigma_I c_I P_I$。基网络设为 4 比特 4 层的硬件高效拟设（HEA），而目标网络为 6, 8, 10 比特的硬件高效拟设线路。采用结构迁移，但只对最上面 4 个比特的参数进行迁移。

3. 海森堡 XXZ 模型的哈密顿量为：
$$H=-J\Sigma_{i}X_iX_{i+1}+Y_iY_{i+1}+hZ_iZ_{i+1}.$$
其中， $J$ 表示最近邻相互作用强度，$h$ 表示 $xy$ 平面和 $z$ 轴耦合的各向异性因子。考虑周期性边界条件，且 $J=1$ 而 $h=2$。基网络设为 4 比特 4 层的哈密顿变分拟设（HVA)（具体搭建方式见 `XXZ_model.ipynb` 中的主程序），而目标网络为 6,8, 12 比特的 HVA 拟设线路，并采用结构迁移方式。

## 量子迁移学习

为将小尺寸线路应用到大尺度线路上，可采用两种迁移方法：网络迁移和结构迁移。其中，网络迁移指的是直接复制多份基网络作为目标网络，而结构迁移则在迁移时，保持网络结构不变。但无论是网络迁移，还是结构迁移，基网络中的最优参数都一起迁移。下面分别是网络迁移和结构迁移的示意图。

![transfer_learning](./src/transfer_learning.jpg)

其中，(a) 表示了 4 比特的 HEA 拟设线路，每一层都由单比特旋转和双比特纠缠门构成，重复多层可有效增加网络的表达性。(b) 展示了根据系统哈密顿量构造的哈密顿变分拟设 (HVA)。(c) 展示了将 4 比特的基线路，以网络迁移的方式，迁移到目标网络上。(d) 为将 2 比特 2 层的基线路，以结构迁移的方式，迁移到 4 比特 4 层的目标网络上。

## 贫瘠高原现象的表示

贫瘠高原现象可由不同初始化方式下，网络参数导数大小来表示。其可用两个指标来进行估计：首先考虑网络第一个有效参数导数的标准差：$\partial_0C$，其中，如果网络为 HEA 拟设，0 指的是第一个 $RX$ 门的参数，如果网络为 HVA 拟设时，0 指的是 $\theta_{1,1}$。另外一个指标为网络所有参数的平均正则范数 $G=\frac{1}{L}\sqrt{\Sigma_{j=1}^{L}(\partial_jC)^2}$ 。

## 自验结果

下图展示了在横场伊辛模型下，采用网络迁移方式，基于基网络构建目标网络中，但采用随机初始化参数方式，$\partial_0C$ 随目标网络比特数的变化情况
（该图通过运行 `Ising_model.ipynb` 中的 `主程序 1` 得到，结果和论文图 3 a 结论一致）。
我们可以看到，贫瘠高原现象的发生。

![Fig3a](./src/Fig3a.png)

下图展示了在横场伊辛模型下，采用网络迁移方式，基于基网络构建目标网络中，分别采用随机初始化 (RR图例) 和使用基网络最优参数初始化 (TT 图例) 两种初始化方式下（基网络最优参数通过执行 `Ising_model.ipynb` 中的 `主程序 0` 得到，并输出到 `Ising_base_net_paras.npy` 中，迁移时自动导入调用），网络所有参数的平均正则范数 $G$ 随目标网络比特数的变化情况。（该图通过运行 `Ising_model.ipynb` 中的 `主程序 2` 得到，结果和论文图 3 d 结论一致）。

![Fig3d](./src/Fig3d.png)

下面两图分别展示了在氢链模型下，采用结构迁移方式，基于基网络构建目标网络中，采用随机初始化 (RR 图例) 与使用基网络最优参数初始化 (TT 图例) 两种初始化方式下（基网络最优参数通过执行 `Hydrogen_model.ipynb` 中的 `主程序 0` 得到，并输出到 `Hydrogen_base_net_paras.npy` 中，迁移时自动导入调用），$\partial_0C$ 随目标网络比特数的变化情况（该图通过运行 `Hydrogen_model.ipynb` 中的 `主程序 1` 得到，结果和论文图 3 b 结论一致）和网络所有参数的平均正则范数 $G$ 随目标网络比特数的变化情况。（该图通过运行 `Hydrogen_model.ipynb` 中的 `主程序 2` 得到，结果和论文图 3 e 结论一致）。

![Fig3b](./src/Fig3b.png)
![Fig3e](./src/Fig3e.png)

下面两图分别展示了在海森堡 XXZ 模型下，采用结构迁移方式，基于基网络构建目标网络中，采用不同参数初始化方式下（  R 表示随机参数层  T 表示最佳参数层。 RT 表示目标网络由随机参数层和最佳参数层交替构成随机初始化，RR、TT 和 TR 类似定义），$\partial_0C$ 随目标网络比特数的变化情况（该图通过运行 `XXZ_model.ipynb` 中的 `主程序 1` 得到，结果和论文图 3 c  结论一致）和网络所有参数的平均正则范数 $G$ 随目标网络比特数的变化情况。（该图通过运行 `XXZ_model.ipynb` 中的 `主程序 2` 得到，结果和论文图 3 f 结论一致）。



![Fig3b](./src/Fig3c.png)
![Fig3e](./src/Fig3f.png)

**注：** 上述每一项数据，都是 100 次采样所得到的平均值。

**结论：** 通过以上复现结果，可见采用量子迁移学习，确实可以缓解贫瘠高原现象。


## 自验环境

硬件：华为云 CouldIDE、 2U8G 20G 存储 、Linux 系统

软件：Mindspore 1.6.0、Mindquantum 0.5.0rc1、numpy 1.21.5

训练超参数：learning rate = 0.05、optimizer = Adam、max_step = 200
